In [ ]:
    print(wrong_contours)
    print()

    for (x, y, w, h) in wrong_contours:
        
        #if (min_w <= w < median_w) or \
        if (w >= min_w and w < median_w) or \
            (h >= min_h and h < median_h):
            #(min_h <= h < median_h):
            wrong_contours.remove((x, y, w, h))
    print(wrong_contours)
    print(min_w, min_h)

In [ ]:
def detect_contours(original_img, edged_img):
    """
    Detects contours in the preprocessed image and filters them based on size, aspect ratio, and area.

    Parameters:
    - original_img: The original input image, used to draw bounding boxes around detected contours.
    - morph_img: The preprocessed image after edge detection and morphological operations.

    Steps:
    NEED TO REWRITE STEPS
    1. Detects contours in the morphological image using OpenCV's `findContours` function.
    2. Sorts the detected contours from left to right using a custom sorting function (`sort_contours`).
    3. Filters the contours based on width, height, aspect ratio, and area to remove unwanted contours.
    4. Draws green rectangles around the filtered contours on the original image to highlight the detected regions.

    Returns:
    - detected_img: The original image with green rectangles drawn around the detected and filtered contours.
    - right_contours : A list of bounding box dimensions (x, y, w, h) for valid contours.
    """
    detected_image = original_img.copy()
    
    # Detect contours
    cnts = cv2.findContours(edged_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    # Sorting contours from left to right
    cnts = sort_contours(cnts, method = "left-to-right")[0]

    all_contours   = []

    for c in cnts:
        x, y, w, h = cv2.boundingRect(c) # Getting a rectangle around a contour
        all_contours.append((x, y, w, h))

    merged_contours = merge_overlapping_boxes(all_contours)

    median_w, median_h, min_w, min_h, max_w, max_h = get_median_min_max_dimensions(merged_contours)

    w_margin = median_w * CONTOUR_W_MARGIN_ERROR
    h_margin = median_h * CONTOUR_H_MARGIN_ERROR

    right_contours, wrong_contours = split_right_wrong_contours(merged_contours, median_w, median_h, w_margin, h_margin)
            
    combined_contours, wrong_contours = process_wrong_contours(wrong_contours, median_w, median_h, w_margin, h_margin)

    right_contours += combined_contours

    #all_c = right_contours + wrong_contours
    #combined_contours, wrong_contours = process_wrong_contours(all_c, median_w, median_h, w_margin, h_margin)

    #all_c_m = merge_overlapping_boxes(all_c)
    right_contours = merge_overlapping_boxes(right_contours)
    '''

    print(wrong_contours)
    print()

    for (x, y, w, h) in wrong_contours[:]:
        
        #if (min_w <= w < median_w) or \
        if (w >= min_w and w <= min_w + w_margin) and \
            (h >= min_h and h <= min_h + h_margin):
            #(min_h <= h < median_h):
            wrong_contours.remove((x, y, w, h))
    print(wrong_contours)
    print(min_w, min_h)
    '''
    
    # Drawing rectangles around each found character
    #ROI_number = 0
    for (x, y, w, h) in right_contours:
        detected_image = cv2.rectangle(detected_image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green rectangles
        #cv2.putText(detected_image, f'({x}, {y})', (x, y), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 1)

        #save_ROI(detected_image, x, y, w, h, ROI_number)
        #ROI_number += 1

    return detected_image, right_contours


In [ ]:
# --- AGAIN ---

    to_merge = right_contours + wrong_contours

    #merged_right_wrong_contours = merge_overlapping_boxes(to_merge)

    new_right, new_wrong = split_right_wrong_contours(to_merge, median_w, median_h, w_margin, h_margin)

    new_combined, new_wrong = process_wrong_contours(new_wrong, median_w, median_h, w_margin, h_margin)

In [4]:
# define the dict of label names
character_dict = {0: '一', 1: '七', 2: '三', 3: '上', 4: '下', 5: '不', 6: '东', 7: '么', 8: '九', 9: '习', 10: '书', 11: '买', 12: '了',
                  13: '二', 14: '五', 15: '些', 16: '亮', 17: '人', 18: '什', 19: '今', 20: '他', 21: '们', 22: '会', 23: '住', 24: '作',
                  25: '你', 26: '候', 27: '做', 28: '儿', 29: '先', 30: '八', 31: '六', 32: '关', 33: '兴', 34: '再', 35: '写', 36: '冷',
                  37: '几', 38: '出', 39: '分', 40: '前', 41: '北', 42: '医', 43: '十', 44: '午', 45: '去', 46: '友', 47: '吃', 48: '同',
                  49: '名', 50: '后', 51: '吗', 52: '呢', 53: '和', 54: '哪', 55: '商', 56: '喂', 57: '喜', 58: '喝', 59: '四', 60: '回',
                  61: '国', 62: '在', 63: '坐', 64: '块', 65: '多', 66: '大', 67: '天', 68: '太', 69: '她', 70: '好', 71: '妈', 72: '姐',
                  73: '子', 74: '字', 75: '学', 76: '客', 77: '家', 78: '对', 79: '小', 80: '少', 81: '岁', 82: '工', 83: '师', 84: '年',
                  85: '店', 86: '开', 87: '影', 88: '很', 89: '怎', 90: '想', 91: '我', 92: '打', 93: '日', 94: '时', 95: '明', 96: '星',
                  97: '昨', 98: '是', 99: '月', 100: '有', 101: '朋', 102: '服', 103: '期', 104: '本', 105: '机', 106: '来', 107: '杯',
                  108: '果', 109: '校', 110: '样', 111: '桌', 112: '椅', 113: '欢', 114: '气', 115: '水', 116: '汉', 117: '没', 118: '漂',
                  119: '火', 120: '点', 121: '热', 122: '爱', 123: '爸', 124: '狗', 125: '猫', 126: '现', 127: '生', 128: '电', 129: '的',
                  130: '看', 131: '睡', 132: '租', 133: '站', 134: '米', 135: '系', 136: '老', 137: '能', 138: '脑', 139: '苹', 140: '茶',
                  141: '菜', 142: '衣', 143: '西', 144: '见', 145: '视', 146: '觉', 147: '认', 148: '识', 149: '话', 150: '语', 151: '说',
                  152: '请', 153: '读', 154: '谁', 155: '谢', 156: '起', 157: '车', 158: '这', 159: '那', 160: '都', 161: '里', 162: '钟',
                  163: '钱', 164: '院', 165: '雨', 166: '零', 167: '面', 168: '飞', 169: '饭', 170: '馆', 171: '高'}

In [6]:
for i in character_dict.values():
    print(i)

一
七
三
上
下
不
东
么
九
习
书
买
了
二
五
些
亮
人
什
今
他
们
会
住
作
你
候
做
儿
先
八
六
关
兴
再
写
冷
几
出
分
前
北
医
十
午
去
友
吃
同
名
后
吗
呢
和
哪
商
喂
喜
喝
四
回
国
在
坐
块
多
大
天
太
她
好
妈
姐
子
字
学
客
家
对
小
少
岁
工
师
年
店
开
影
很
怎
想
我
打
日
时
明
星
昨
是
月
有
朋
服
期
本
机
来
杯
果
校
样
桌
椅
欢
气
水
汉
没
漂
火
点
热
爱
爸
狗
猫
现
生
电
的
看
睡
租
站
米
系
老
能
脑
苹
茶
菜
衣
西
见
视
觉
认
识
话
语
说
请
读
谁
谢
起
车
这
那
都
里
钟
钱
院
雨
零
面
飞
饭
馆
高


In [1]:
import tkinter as tk

class ToolTip:
    def __init__(self, widget, text):
        self.widget = widget
        self.text = text
        self.tip_window = None
        self.widget.bind("<Enter>", self.show_tooltip)
        self.widget.bind("<Leave>", self.hide_tooltip)

    def show_tooltip(self, event=None):
        # Создаем окно подсказки
        if self.tip_window or not self.text:
            return
        x, y, cx, cy = self.widget.bbox("insert")
        x += self.widget.winfo_rootx() + 25
        y += self.widget.winfo_rooty() + 20
        self.tip_window = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(True)  # Убираем рамки окна
        tw.wm_geometry(f"+{x}+{y}")

        # Создаем метку с текстом
        label = tk.Label(
            tw, 
            text=self.text, 
            justify="left",
            background="lightyellow", 
            relief="solid", 
            borderwidth=1,
            font=("tahoma", "10", "normal")
        )
        label.pack(ipadx=5, ipady=3)

    def hide_tooltip(self, event=None):
        if self.tip_window:
            self.tip_window.destroy()
            self.tip_window = None


# Пример использования
root = tk.Tk()
root.title("Пример всплывающей подсказки")

btn = tk.Button(root, text="Наведи мышкой")
btn.pack(pady=20)

# Добавляем подсказку к кнопке
tooltip = ToolTip(btn, "Это кнопка!")

root.mainloop()
